# Train Base Models

### Purpose:

Repeat NB 3 but use the one-hot encoded version of the dataset

### Result:
One hot encoding helped a decent amount, RMSLE 1.314 -> 1.16918

BUT, median heuristic still outperforms with a score of 1.10815

In [ ]:
%pip install -r ../../requirements.txt

# load dataset

In [3]:
base_dataset = pd.read_csv('../data/baseline_dataset_one_hot.csv')
test_dataset = pd.read_csv('../data/raw_dataset/test.csv')

In [4]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR


In [5]:
# Explicitly define the target column
target_column = 'Premium Amount'

# Separate features (X) and target (y)
X = base_dataset.drop(columns=[target_column])
y = base_dataset[target_column]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define baseline models
models = {
    "Linear Regression": LinearRegression(),
    "Ridge Regression": Ridge(),
    "Decision Tree": DecisionTreeRegressor(),
    # "Random Forest": RandomForestRegressor(n_estimators=100),
    # "Support Vector Regressor": SVR()
}

# Train and evaluate each model
results = []

for name, model in models.items():
    print(f"begin {name} training")
    # Train the model
    model.fit(X_train, y_train)
    
    # Make predictions
    y_pred = model.predict(X_test)
    
    # Evaluate performance
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    # Store results
    results.append({
        "Model": name,
        "MSE": mse,
        "MAE": mae,
        "R^2": r2
    })
    print(f"{name} training complete")

    

# Display results
results_df = pd.DataFrame(results).sort_values(by="R^2", ascending=False)
results_df

begin Linear Regression training
Linear Regression training complete
begin Ridge Regression training
Ridge Regression training complete
begin Decision Tree training
Decision Tree training complete


,Model,MSE,MAE,R^2
1,Ridge Regression,7.514585e+05,671.019662,0.003282
0,Linear Regression,7.514585e+05,671.019662,0.003282
2,Decision Tree,1.522074e+06,897.565982,-1.018846


In [7]:
#minimally process the test dataset to get model predictions

#convert the policy start time to duration in mins
test_dataset['Policy Start Date'] = pd.to_datetime(test_dataset['Policy Start Date'])
test_dataset['Policy Duration Mins'] = ((pd.Timestamp.now() - test_dataset['Policy Start Date']).dt.total_seconds())/60
test_dataset = test_dataset.drop(columns=['Policy Start Date'])

#do label encoding
categorical_cols = [test_dataset.columns[i] for i, x in enumerate(test_dataset.dtypes) if x == 'object']
test_dataset[categorical_cols] = test_dataset[categorical_cols].astype('category')
# Convert categorical to one hot encodings
test_dataset = pd.get_dummies(test_dataset, drop_first=True)
#fill nulls with median values
test_dataset = test_dataset.fillna(test_dataset.median())


features = test_dataset.iloc[:,1:]

In [9]:
#generate results and submit to competition
results_directory = "../results"

for name, model in models.items():
    
    y_pred = model.predict(features)

    results = pd.DataFrame({
        'id': test_dataset['id'],  
        'Premium Amount': y_pred   
    })

    filename = f"{name}_baseline_one_hot.csv"
    results_full_path = os.path.join(results_directory,filename)
    
    results.to_csv(results_full_path, index=False)

    submission_comment = f"baseline: {name} + minimally processed one-hot encoded dataset"
    submit(filename,submission_comment) #custom function
    # print(f"successfully submitted baseline {name}")


Competition Name: playground-series-s4e12
Full Submission File Path: ../results\Linear Regression_baseline_one_hot.csv


100%|██████████| 21.2M/21.2M [00:18<00:00, 1.19MB/s]


Submission to 'playground-series-s4e12' successful!
Competition Name: playground-series-s4e12
Full Submission File Path: ../results\Ridge Regression_baseline_one_hot.csv


100%|██████████| 21.2M/21.2M [00:23<00:00, 938kB/s] 


Submission to 'playground-series-s4e12' successful!
Competition Name: playground-series-s4e12
Full Submission File Path: ../results\Decision Tree_baseline_one_hot.csv


100%|██████████| 11.7M/11.7M [00:10<00:00, 1.16MB/s]


Error during submission: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Content-Type': 'application/json', 'Date': 'Fri, 20 Dec 2024 21:23:12 GMT', 'Access-Control-Allow-Credentials': 'true', 'Access-Control-Allow-Origin': '*', 'Set-Cookie': 'ka_sessionid=8280b94f9f57a2bbcc6ae8833076ad2f; max-age=2626560; path=/, GCLB=CPPr3eeR9a2pShAD; path=/; HttpOnly', 'Vary': 'Accept-Encoding', 'X-Kaggle-MillisecondsElapsed': '81', 'X-Kaggle-RequestId': '76e67705afce2c0954954273039ebdcf', 'X-Kaggle-ApiVersion': '1.6.17', 'X-Kaggle-HubVersion': '0.3.6', 'X-Frame-Options': 'SAMEORIGIN', 'Strict-Transport-Security': 'max-age=63072000; includeSubDomains; preload', 'Content-Security-Policy': "object-src 'none'; script-src 'nonce-m/bUG9ngxI+m2DTFWdx2Hg==' 'report-sample' 'unsafe-inline' 'unsafe-eval' 'strict-dynamic' https: http:; base-uri 'none'; report-uri https://csp.withgoogle.com/csp/kaggle/20201130; frame-src 'self' https://www.kaggleusercontent.com https://www.youtube.com/embed/ 